In [1]:
import pickle
from collections import namedtuple

import pandas as pd
from arguing_lexicon import ArguingLexiconParser
from tqdm import tqdm_notebook as tqdm

input_data_csv = "data/captions_metadata.csv"

In [2]:
# Lazy data reader into DataFrame
transcripts_reader = pd.read_csv("data/captions_metadata.csv", chunksize=10)

# Dummy class for convenience and speed
Doc = namedtuple("Doc", ["text"])

# Util function to write lazy chunks back to disk
output_columns = ["id", "content", "date", "title", "unknown", "channel", "fragments", "argument_fragments",
                  "argument_labels", "argument_content"]
arguments_path = "data/captions_arguments.csv"
def write_to_disk(chunk):
    with open(arguments_path, "a", encoding="utf-8") as arguments_file:
        chunk.to_csv(arguments_file, index=False)

# Arguing lexixon parser
parser = ArguingLexiconParser()


In [ ]:
for ix, transcripts in enumerate(tqdm(transcripts_reader)):
    if ix <= 1183:
        continue
    arguments_frame = pd.DataFrame(columns=output_columns)
    for ix, transcript in transcripts.iterrows():
        content = str(transcript["content"]).split("\n")
        labels = []
        arguments = []
        argument_fragments = 0
        for con in content:
            doc = Doc(con)
            matches = list(parser.get_arguing_matches(doc))
            if len(matches):
                argument_fragments += 1
            lbls = []
            args = []
            for label, match in matches:
                lbls.append(label)
                args.append(match.group(0))
            labels.append("\t".join(lbls))
            arguments.append("\t".join(args))
        argument_serie = pd.Series(data={
            "id": transcript["id"],
            "content": transcript["content"],
            "date": transcript["date"],
            "title": transcript["title"],
            "unknown": transcript["unknown"],
            "channel": transcript["channel"],
            "fragments": len(content),
            "argument_fragments": argument_fragments,
            "argument_labels": "\n".join(labels),
            "argument_content": "\n".join(arguments)
        })

        arguments_frame = arguments_frame.append(argument_serie, ignore_index=True)
    write_to_disk(arguments_frame)